# Kayak Project: Welcome!

## Import list of cities and create the dataframe associated

In [1]:
# Create the city list

cities = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle",
]


In [2]:
import pandas as pd

# Creating a dataframe from the list of cities
df_cities = pd.DataFrame(data=cities, columns=["city"])
df_cities = df_cities.reset_index()
df_cities = df_cities.rename(columns={"index": "id"})
df_cities


,id,city
0,0,Mont Saint Michel
1,1,St Malo
2,2,Bayeux
3,3,Le Havre
4,4,Rouen
5,5,Paris
6,6,Amiens
7,7,Lille
8,8,Strasbourg
9,9,Chateau du Haut Koenigsbourg


## Nominatim API : get coordinates of cities => https://nominatim.org/release-docs/latest/

### Try a city with blank space in name to see if formatting needed:

In [3]:
import requests

params = {"city": "La Rochelle", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


[{'place_id': 281822562,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 117858,
  'boundingbox': ['46.1331804', '46.1908971', '-1.2419231', '-1.111097'],
  'lat': '46.1591126',
  'lon': '-1.1520434',
  'display_name': 'La Rochelle, Charente-Maritime, Nouvelle-Aquitaine, France métropolitaine, 17000, France',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.9114837096874572,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'},
 {'place_id': 282096286,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 1215878,
  'boundingbox': ['47.7388479', '47.7647325', '5.7060641', '5.7454734'],
  'lat': '47.7470598',
  'lon': '5.7321403',
  'display_name': 'La Rochelle, Vesoul, Haute-Saône, Bourgogne-Franche-Comté, France métropolitaine, 70120, France',
  'class': 'boundary',
  'type'

#### That doesn't work... After Google search, 'Castellane'

### Since I don't know 'Chateau du Haut Koenigsbourg', I'm testing if Nominatim knows it

In [4]:
import requests

params = {"city": "Chateau du Haut Koenigsbourg",
          "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


[{'place_id': 49750339,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 4245068168,
  'boundingbox': ['48.2494726', '48.2495726', '7.3454423', '7.3455423'],
  'lat': '48.2495226',
  'lon': '7.3454923',
  'display_name': 'Château du Haut-Kœnigsbourg, Orschwiller, Sélestat-Erstein, Bas-Rhin, Grand Est, France métropolitaine, 67600, France',
  'class': 'place',
  'type': 'isolated_dwelling',
  'importance': 0.51}]

### 'Gorges du Verdon' isn't a city but a canyon, I check if Nominatim works with or not

In [5]:
import requests

params = {"city": "Gorges du Verdon", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


[]

Doesnt' works... Google search : 'Castellane' will be use for Gorges du Verdon

### Ariege is a department and not a City, its prefecture is 'Foix'. We'll use this city

### Request API

In [6]:
# Creating a copy of df_cities to store the coordinates from Nominatim API

df_gps = df_cities.copy(deep=True)
lat_list = []
lon_list = []

for i in cities:
    print(f"Request for city: {i}")
    params = {"city": i, "country": "France", "format": "json"}
    # No 'Gorges du Verdon' city, replacing by 'Castellane'
    if i == "Gorges du Verdon":
        i = "Castellane"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    # 'Ariege' not a city, using the prefecture instead -> 'Foix'
    elif i == "Ariege":
        i = "Foix"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    else:
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?", params).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])

# Adding the coordinates to the dataframe
df_gps["lat"] = lat_list
df_gps["lon"] = lon_list


Request for city: Mont Saint Michel
Request for city: St Malo
Request for city: Bayeux
Request for city: Le Havre
Request for city: Rouen
Request for city: Paris
Request for city: Amiens
Request for city: Lille
Request for city: Strasbourg
Request for city: Chateau du Haut Koenigsbourg
Request for city: Colmar
Request for city: Eguisheim
Request for city: Besancon
Request for city: Dijon
Request for city: Annecy
Request for city: Grenoble
Request for city: Lyon
Request for city: Gorges du Verdon
Request for city: Bormes les Mimosas
Request for city: Cassis
Request for city: Marseille
Request for city: Aix en Provence
Request for city: Avignon
Request for city: Uzes
Request for city: Nimes
Request for city: Aigues Mortes
Request for city: Saintes Maries de la mer
Request for city: Collioure
Request for city: Carcassonne
Request for city: Ariege
Request for city: Toulouse
Request for city: Montauban
Request for city: Biarritz
Request for city: Bayonne
Request for city: La Rochelle


In [7]:
df_gps


,id,city,lat,lon
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658
5,5,Paris,48.8588897,2.3200410217200766
6,6,Amiens,49.8941708,2.2956951
7,7,Lille,50.6365654,3.0635282
8,8,Strasbourg,48.584614,7.7507127
9,9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923


In [8]:
df_gps.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      35 non-null     int64 
 1   city    35 non-null     object
 2   lat     35 non-null     object
 3   lon     35 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


## OpenWeather : get weather of the week => https://openweathermap.org

In [9]:
df_gps.head()


,id,city,lat,lon
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


In [10]:
parameters = {
    "lat": 48.649518,
    "lon": -2.0260409,
    "exclude": "current,minutely,hourly",
    "units": "metric",
    "appid": "aa423e6694bf72625fe1fe31544949dc",
    "lang": "fr",
}

r = requests.get(
    "https://api.openweathermap.org/data/2.5/onecall", params=parameters
).json()

r


{'lat': 48.6495,
 'lon': -2.026,
 'timezone': 'Europe/Paris',
 'timezone_offset': 7200,
 'daily': [{'dt': 1652097600,
   'sunrise': 1652070945,
   'sunset': 1652124802,
   'moonrise': 1652095440,
   'moonset': 1652062260,
   'moon_phase': 0.25,
   'temp': {'day': 21.74,
    'min': 11.47,
    'max': 24.43,
    'night': 15.23,
    'eve': 20.9,
    'morn': 12.33},
   'feels_like': {'day': 21.35, 'night': 14.98, 'eve': 20.59, 'morn': 11.89},
   'pressure': 1023,
   'humidity': 53,
   'dew_point': 11.75,
   'wind_speed': 4.17,
   'wind_deg': 71,
   'wind_gust': 5.81,
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'nuageux',
     'icon': '04d'}],
   'clouds': 56,
   'pop': 0,
   'uvi': 5.92},
  {'dt': 1652184000,
   'sunrise': 1652157258,
   'sunset': 1652211287,
   'moonrise': 1652186100,
   'moonset': 1652149980,
   'moon_phase': 0.29,
   'temp': {'day': 17.93,
    'min': 12.29,
    'max': 17.98,
    'night': 12.29,
    'eve': 14.13,
    'morn': 12.78},
   'feels_lik

In [11]:
r["daily"][3:]  # Weather in 3 days


[{'dt': 1652356800,
  'sunrise': 1652329887,
  'sunset': 1652384254,
  'moonrise': 1652367780,
  'moonset': 1652324940,
  'moon_phase': 0.36,
  'temp': {'day': 14.27,
   'min': 9.85,
   'max': 14.27,
   'night': 11.04,
   'eve': 13.14,
   'morn': 10.67},
  'feels_like': {'day': 13.53, 'night': 10.39, 'eve': 12.36, 'morn': 10.14},
  'pressure': 1021,
  'humidity': 68,
  'dew_point': 8.31,
  'wind_speed': 4.97,
  'wind_deg': 317,
  'wind_gust': 6.95,
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'couvert',
    'icon': '04d'}],
  'clouds': 100,
  'pop': 0.2,
  'uvi': 6.2},
 {'dt': 1652443200,
  'sunrise': 1652416204,
  'sunset': 1652470737,
  'moonrise': 1652458860,
  'moonset': 1652412360,
  'moon_phase': 0.4,
  'temp': {'day': 15.36,
   'min': 9.81,
   'max': 15.36,
   'night': 11.73,
   'eve': 13.59,
   'morn': 10.49},
  'feels_like': {'day': 14.75, 'night': 11.33, 'eve': 13.09, 'morn': 9.84},
  'pressure': 1023,
  'humidity': 69,
  'dew_point': 9.33,
  'wind_speed

In [12]:
day3 = r["daily"][3]  # Weather in 3 days

# desciption of the main weather in 3 days to St Malo
day3["weather"][0]["main"]


'Clouds'

In [13]:
import numpy as np

df_weather = df_gps.copy(deep=True)
my_api_key = "aa423e6694bf72625fe1fe31544949dc"

temperatures_list = []
rain_list = []
weather_list = []

days = list(range(1, 8))

for i in df_weather.itertuples():
    lat = i.lat
    lon = i.lon

    parameters = {
        "lat": {lat},
        "lon": {lon},
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": "aa423e6694bf72625fe1fe31544949dc",
        "lang": "fr",
    }

    r = requests.get(
        f"https://api.openweathermap.org/data/2.5/onecall?", parameters
    ).json()
    forecast_7days = r["daily"][
        1:
    ]  # Getting the weather data for the next 7 days, first item is the current weather, which we don't want here
    temperatures = [int(d["feels_like"]["day"]) for d in forecast_7days]
    rain = [int(d["pop"] * 100) for d in forecast_7days]
    weather = [str(d["weather"][0]["main"]) for d in forecast_7days]
    temperatures_list.append(temperatures)
    rain_list.append(rain)
    weather_list.append(weather)

df_weather["jour_+x"] = [days for _ in range(len(df_weather))]
df_weather["temperature_ressentie"] = temperatures_list
df_weather["probabilite_de_pluie"] = rain_list
df_weather["meteo_principale"] = weather_list

df_weather


,id,city,lat,lon,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,"[1, 2, 3, 4, 5, 6, 7]","[19, 19, 14, 15, 18, 22, 24]","[0, 43, 21, 0, 0, 0, 0]","[Clouds, Clear, Clouds, Clouds, Clear, Clouds,..."
1,1,St Malo,48.649518,-2.0260409,"[1, 2, 3, 4, 5, 6, 7]","[17, 17, 13, 14, 15, 18, 20]","[0, 55, 20, 0, 0, 0, 0]","[Clouds, Rain, Clouds, Clouds, Clear, Clouds, ..."
2,2,Bayeux,49.2764624,-0.7024738,"[1, 2, 3, 4, 5, 6, 7]","[18, 18, 15, 16, 16, 19, 21]","[0, 53, 37, 0, 0, 0, 0]","[Clouds, Rain, Clouds, Clouds, Clear, Clear, C..."
3,3,Le Havre,49.4938975,0.1079732,"[1, 2, 3, 4, 5, 6, 7]","[15, 14, 13, 14, 14, 18, 19]","[0, 42, 36, 0, 0, 0, 0]","[Clouds, Rain, Clouds, Clouds, Clouds, Clear, ..."
4,4,Rouen,49.4404591,1.0939658,"[1, 2, 3, 4, 5, 6, 7]","[20, 19, 17, 16, 18, 20, 22]","[0, 43, 18, 0, 0, 0, 0]","[Clouds, Rain, Clouds, Clouds, Clouds, Clear, ..."
5,5,Paris,48.8588897,2.3200410217200766,"[1, 2, 3, 4, 5, 6, 7]","[23, 22, 19, 19, 20, 22, 24]","[0, 3, 4, 0, 0, 0, 0]","[Clouds, Clear, Clouds, Clouds, Clear, Clouds,..."
6,6,Amiens,49.8941708,2.2956951,"[1, 2, 3, 4, 5, 6, 7]","[20, 19, 17, 16, 16, 19, 22]","[0, 54, 39, 0, 0, 0, 0]","[Clouds, Rain, Clouds, Clouds, Clouds, Clouds,..."
7,7,Lille,50.6365654,3.0635282,"[1, 2, 3, 4, 5, 6, 7]","[21, 18, 17, 16, 18, 19, 22]","[0, 85, 86, 0, 0, 0, 0]","[Clouds, Rain, Rain, Clouds, Clouds, Clouds, C..."
8,8,Strasbourg,48.584614,7.7507127,"[1, 2, 3, 4, 5, 6, 7]","[22, 24, 23, 19, 20, 21, 23]","[0, 17, 71, 81, 0, 0, 0]","[Clouds, Clouds, Rain, Rain, Clouds, Clear, Cl..."
9,9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923,"[1, 2, 3, 4, 5, 6, 7]","[19, 22, 20, 15, 18, 18, 19]","[0, 13, 92, 96, 0, 0, 0]","[Clouds, Clouds, Rain, Rain, Clouds, Clear, Cl..."


In [14]:
df_weather.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     35 non-null     int64 
 1   city                   35 non-null     object
 2   lat                    35 non-null     object
 3   lon                    35 non-null     object
 4   jour_+x                35 non-null     object
 5   temperature_ressentie  35 non-null     object
 6   probabilite_de_pluie   35 non-null     object
 7   meteo_principale       35 non-null     object
dtypes: int64(1), object(7)
memory usage: 2.3+ KB


In [15]:
df_weather[["lat", "lon"]] = df_weather[["lat", "lon"]].astype(float)


In [16]:
df = df_weather.apply(pd.Series.explode)


In [17]:
df


,id,city,lat,lon,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale
0,0,Mont Saint Michel,48.635954,-1.511460,1,19,0,Clouds
0,0,Mont Saint Michel,48.635954,-1.511460,2,19,43,Clear
0,0,Mont Saint Michel,48.635954,-1.511460,3,14,21,Clouds
0,0,Mont Saint Michel,48.635954,-1.511460,4,15,0,Clouds
0,0,Mont Saint Michel,48.635954,-1.511460,5,18,0,Clear
...,...,...,...,...,...,...,...,...
34,34,La Rochelle,46.159113,-1.152043,3,18,5,Clouds
34,34,La Rochelle,46.159113,-1.152043,4,18,3,Clouds
34,34,La Rochelle,46.159113,-1.152043,5,20,0,Clear
34,34,La Rochelle,46.159113,-1.152043,6,23,1,Clouds


In [18]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 245 entries, 0 to 34
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     245 non-null    int64  
 1   city                   245 non-null    object 
 2   lat                    245 non-null    float64
 3   lon                    245 non-null    float64
 4   jour_+x                245 non-null    object 
 5   temperature_ressentie  245 non-null    object 
 6   probabilite_de_pluie   245 non-null    object 
 7   meteo_principale       245 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 17.2+ KB


In [19]:
df[["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]] = df[["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]].astype(int)


In [24]:
import plotly.express as px

fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    hover_name="city",
    zoom=4,
    hover_data=["meteo_principale", "probabilite_de_pluie", "temperature_ressentie"],
    color="temperature_ressentie",
    color_continuous_scale="thermal",
    mapbox_style="carto-positron",
)
fig.show()
